In [8]:
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
import os
from numpy import nan

In [9]:
# Creating list of taxonomy based on the requirements from excel file
taxonomy_excel = pd.read_excel("/home/rszczygielski/bioinf/magisterka/geneBank/results/taxonomy1.xlsx")
array_of_values = taxonomy_excel.values
list_of_values = []
for row_array in array_of_values:
    row_array = row_array[~pd.isnull(row_array)]
    row_list = list(row_array)
    list_of_values.append(row_list)

taxonomy_row_list = []
for elem in list_of_values:
    new_elem = list(map(str.strip, elem))
    taxonomy_row_list.append(new_elem)
taxonomy_row_list[0]

["'Chordata'",
 "'Craniata'",
 "'Vertebrata'",
 "'Chondrichthyes'",
 "'Elasmobranchii'"]

In [10]:
mitochondrion_type = "2"
main_file_path = f"/home/rszczygielski/bioinf/magisterka/geneBank/results/mitochondrion_{mitochondrion_type}.txt"

df_masters = pd.read_csv(main_file_path,\
    header=0,\
    delimiter='\t',\
    usecols=["ACCESSION", "ORGANISM","TAXONOMY", "PREVIOUS_GENE","CONTROL_REGION", "NEXT_GENE"],\
    index_col="ACCESSION")

df_taxonomy = df_masters[["TAXONOMY", "PREVIOUS_GENE","CONTROL_REGION", "NEXT_GENE"]]
df_taxonomy.shape

(3766, 4)

In [11]:
# TEST
# test_cr = df_taxonomy["CONTROL_REGION"].to_list()
# for cr in test_cr:
#     splited_row = cr.split()
#     if splited_row[-1] != "None":
#         print(cr)

In [12]:
import re
# pd.set_option('display.max_rows', None)

# bool(re.search(r'\d', list_of_bruckets[1]) returns true if any digit is in the string

def get_specific_previous_and_next_gene_info(gene):
    """Method returns transformed gene specific info for a given string

    Args:
        gene (str): gene information

    Returns:
        str: flitrated specific gene type
    """
    list_of_bruckets = list(re.findall(r'\[.*?\]', gene))
    if len(list_of_bruckets) == 1 or len(list_of_bruckets) > 2 or bool(re.search(r'\d', list_of_bruckets[1])):
        result = gene.split()[0]
    else:
        result = list_of_bruckets[1]
        if "[" in result:
            result = re.findall(r"'([^']*)'", result)[0]
    return result

def get_specific_control_region_info(control_region):
    """Method gets name of a control region from control region information string

    Args:
        control_region (string): control region gene info

    Returns:
        str: flitrated control region name
    """
    return control_region.split()[0]

df_taxonomy["PREVIOUS_GENE"] = df_taxonomy["PREVIOUS_GENE"].apply(lambda value: get_specific_previous_and_next_gene_info(value))
df_taxonomy["CONTROL_REGION"] = df_taxonomy["CONTROL_REGION"].apply(lambda value: get_specific_control_region_info(value))
df_taxonomy["NEXT_GENE"] = df_taxonomy["NEXT_GENE"].apply(lambda value: get_specific_previous_and_next_gene_info(value))

# test = df_taxonomy["PREVIOUS_GENE"].apply(lambda value: get_specific_previous_and_next_gene_info(value))

display(df_taxonomy)

/tmp/ipykernel_2612/3228293116.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_taxonomy["PREVIOUS_GENE"] = df_taxonomy["PREVIOUS_GENE"].apply(lambda value: get_specific_previous_and_next_gene_info(value))
/tmp/ipykernel_2612/3228293116.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_taxonomy["CONTROL_REGION"] = df_taxonomy["CONTROL_REGION"].apply(lambda value: get_specific_control_region_info(value))
/tmp/ipykernel_2612/3228293116.py:37: SettingWithCopyWarning: 
A value is trying to be set o

,TAXONOMY,PREVIOUS_GENE,CONTROL_REGION,NEXT_GENE
ACCESSION,,,,
NC_052749.1,"['Eukaryota', 'Metazoa', 'Ecdysozoa', 'Arthrop...",rRNA,D-loop,tRNA-Ile
NC_052688.1,"['Eukaryota', 'Metazoa', 'Chordata', 'Craniata...",tRNA-Pro,D-loop,tRNA-Phe
NC_052715.1,"['Eukaryota', 'Metazoa', 'Ecdysozoa', 'Arthrop...",rRNA,D-loop,tRNA-Ile
NC_052710.1,"['Eukaryota', 'Metazoa', 'Chordata', 'Craniata...",tRNA-Pro,D-loop,tRNA-Phe
NC_052687.1,"['Eukaryota', 'Metazoa', 'Ecdysozoa', 'Arthrop...",rRNA,D-loop,tRNA-Gln
NC_052711.1,"['Eukaryota', 'Metazoa', 'Chordata', 'Craniata...",tRNA-Glu,D-loop,tRNA-Phe
NC_052907.1,"['Eukaryota', 'Metazoa', 'Ecdysozoa', 'Arthrop...",rRNA,D-loop,tRNA-Ile
NC_052734.1,"['Eukaryota', 'Metazoa', 'Ecdysozoa', 'Arthrop...",rRNA,D-loop,tRNA-Ile
NC_052726.1,"['Eukaryota', 'Metazoa', 'Ecdysozoa', 'Arthrop...",rRNA,D-loop,tRNA-Ile


In [13]:
# display(df_taxonomy)
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


general_table = pd.DataFrame()
for row_excel_taxonomy in enumerate(taxonomy_row_list):
    joined_values_entry = ", ".join(row_excel_taxonomy[1])
    aggragated_taxonomy = df_taxonomy.where(df_taxonomy["TAXONOMY"].str.contains(joined_values_entry)).dropna()
    general_table = pd.concat([general_table, aggragated_taxonomy])

general_table.to_excel(f"/home/rszczygielski/bioinf/magisterka/geneBank/results/General_info_mitochondrion_{mitochondrion_type}.xlsx")

In [14]:
general_table.count()
len(general_table["TAXONOMY"].unique())


TAXONOMY          3520
PREVIOUS_GENE     3520
CONTROL_REGION    3520
NEXT_GENE         3520
dtype: int64

2168